In [49]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [50]:
start_case = 500
case_num = 500

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]


def objective(trial: optuna.trial.Trial) -> float:
    min_k = 4
    max_k = 5.5
    k_p = 0.85
    start_step = 0.75
    end_step = 1.8
    step_cnt = 5
    v_prob_mean_w = 1.460
    v_prob_min_var = 0.027
    p_max = 0.9
    p_min = 0.4

    # min_k = trial.suggest_float("max_k", low=1.5, high=4.5)
    # max_k = trial.suggest_float("min_k", low=5.0, high=7.0)
    # k_p = trial.suggest_float("k_p", low=0.5, high=2.0)
    # start_step = trial.suggest_float("start_step", low=0.2, high=1.5)
    # end_step = trial.suggest_float("end_step", low=max(1.2, start_step + 0.1), high=3.2)
    # step_cnt = trial.suggest_int("step_cnt", low=2, high=12)
    # v_prob_mean_w = trial.suggest_float("v_prob_mean_w", low=0.0, high=3.0)
    # v_prob_min_var = trial.suggest_float("v_prob_min_var", low=0.01, high=0.5)
    # p_max = trial.suggest_float("p_max", low=0.5, high=0.99)
    # p_min = trial.suggest_float("p_min", low=0.1, high=min(0.9, p_max + 0.01))


    delta_max_dist = trial.suggest_int("delta_max_dist", low=1, high=3)
    neighbor_p = trial.suggest_float("neighbor_p", low=1., high=3.)
    duplicate_min_d = trial.suggest_int("duplicate_min_d", low=1, high=5)
    step_p = trial.suggest_float("step_p", low=0.5, high=2.)
    out_step_size = trial.suggest_float("out_step_size", low=50., high=200.)
    start_temp_coef = trial.suggest_float("start_temp_coef", low=1e2, high=1e4)
    end_temp_coef = trial.suggest_float("end_temp_coef", low=1e4+1, high=1e6)

    args = " ".join(
        map(
            str,
            [
                min_k,
                max_k,
                k_p,
                start_step,
                end_step,
                step_cnt,
                v_prob_mean_w,
                v_prob_min_var,
                p_max,
                p_min,
                delta_max_dist,
                neighbor_p,
                duplicate_min_d,
                step_p,
                out_step_size,
                start_temp_coef,
                end_temp_coef
            ],
        )
    )

    solver_path = "./target/release/ahc030"
    solver_cmd = f"./tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-anneal-01-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evaluate_overall_relative_score(
        ignore_solver_prefix="optuna-anneal-01-"
    )
    return score_df.relative_score.mean()


subprocess.run(
    "cargo build --features local --release --manifest-path=../Cargo.toml", shell=True
)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc030.db",
    study_name="tune-anneal-02",
    load_if_exists=True,
)
print("start optimize")
study.enqueue_trial(
    {
        "min_k": 4.0,
        "max_k": 5.5,
        "k_p": 0.85,
        "start_step": 0.75,
        "end_step": 1.8,
        "step_cnt": 5,
        "v_prob_mean_w": 1.0,
        "v_prob_min_var": 0.1,
        "p_max": 0.9,
        "p_min": 0.5,
        "delta_max_dist": 2,
        "neighbor_p": 2.,
        "duplicate_min_d": 1,
        "step_p": 1.0,
        "out_step_size": 100.0,
        "start_temp_coef": 1e3,
        "end_temp_coef": 1e5,
    }
)
study.optimize(objective, n_trials=100, show_progress_bar=True)
study.best_params

    Finished release [optimized] target(s) in 0.01s
[I 2024-02-19 12:33:44,158] A new study created in RDB with name: tune-anneal-02


start optimize


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-02-19 12:34:51,436] Trial 0 finished with value: 0.47468504827740327 and parameters: {'delta_max_dist': 2, 'neighbor_p': 2.0, 'duplicate_min_d': 1, 'step_p': 1.0, 'out_step_size': 100.0, 'start_temp_coef': 1000.0, 'end_temp_coef': 100000.0}. Best is trial 0 with value: 0.47468504827740327.
[I 2024-02-19 12:35:58,715] Trial 1 finished with value: 0.3972344514080676 and parameters: {'delta_max_dist': 2, 'neighbor_p': 2.9513793481304513, 'duplicate_min_d': 4, 'step_p': 1.8563271555112155, 'out_step_size': 177.32376842179178, 'start_temp_coef': 3207.8509165162545, 'end_temp_coef': 262817.82791369304}. Best is trial 0 with value: 0.47468504827740327.
[I 2024-02-19 12:37:19,738] Trial 2 finished with value: 0.3780410731344945 and parameters: {'delta_max_dist': 2, 'neighbor_p': 2.9103571666669956, 'duplicate_min_d': 3, 'step_p': 0.9856006924038068, 'out_step_size': 140.40842440770595, 'start_temp_coef': 6280.566064171002, 'end_temp_coef': 460605.6518207593}. Best is trial 0 with value

KeyboardInterrupt: 

In [51]:
study = optuna.load_study(
    storage="sqlite:///ahc030.db",
    study_name="tune-anneal-02",
)
study.best_params

{'delta_max_dist': 2,
 'duplicate_min_d': 1,
 'end_temp_coef': 43417.675534323964,
 'neighbor_p': 2.0130480815748517,
 'out_step_size': 100.41986846188436,
 'start_temp_coef': 722.1025458477571,
 'step_p': 1.1213463590452353}

In [54]:
optuna.visualization.plot_slice(study)

In [55]:
optuna.visualization.plot_optimization_history(study)